#### manage steams data

In [2]:
import pandas as pd
from hana_ml import dataframe

try:
    import configparser
except ImportError:
    import ConfigParser as configparser

settings = configparser.ConfigParser()
settings.read("../../config/e2edata.ini")

url = settings.get("hana", "url")
port = settings.getint("hana", "port")
user = settings.get("hana", "user")
pwd = settings.get("hana", "passwd")

connection_context = dataframe.ConnectionContext(url, port, user, pwd)

In [ ]:
connection_context.create_table("#LOB_STREAMING_TEST", table_structure={"id": "int", "img": "blob", "img2": "blob"})

In [ ]:

with open('./images/Crime-Rate-and-Median-House-Prices.png', 'rb') as f:
    img = f.read()
with open('./images/Inflation_Adjusted_Housing_Prices_1890_2006.jpg', 'rb') as f:
    img2 = f.read()
connection_context.upsert_streams_data(table_name="#LOB_STREAMING_TEST", key="id", data={"id":1, "img":img, "img2":img2})

In [ ]:
connection_context.table("#LOB_STREAMING_TEST").collect()

In [ ]:
connection_context.update_streams_data(table_name="#LOB_STREAMING_TEST", key="id", data={"id":1, "img":img2, "img2":img})

In [ ]:
connection_context.table("#LOB_STREAMING_TEST").collect()

In [ ]:
conn = connection_context.connection

conn.setautocommit(True)
cursor = conn.cursor()
cursor.execute("select \"img\" from #LOB_STREAMING_TEST where \"id\" = :id", {"id": 1}) 
out_blob = cursor.fetchone(True)[0]
out_img = bytearray()
while True:
    data = out_blob.read(size=200)
    if data is None:
        print("Done.")
        break
    else:
        print("Reading more data...")
        out_img.extend(data.tobytes())

In [ ]:
import PIL.Image as Image
from io import BytesIO
image = Image.open(BytesIO(out_img))

In [ ]:
display(image)

#### manage temporary tables

In [ ]:
connection_context.get_temporary_tables(list_other_connections=False)

In [ ]:
connection_context.get_connection_id()

In [ ]:
connection_context.clean_up_temporary_tables()
connection_context.get_temporary_tables()

#### Manual Pipeline integration with auto-ml

In [ ]:
from hana_ml.algorithms.pal.pipeline import Pipeline
from hana_ml.algorithms.pal.decomposition import PCA
from hana_ml.algorithms.pal.preprocessing import Imputer
from hana_ml.algorithms.pal.trees import HybridGradientBoostingClassifier
from hana_ml.algorithms.pal.utility import DataSets, Settings
my_pipeline = Pipeline([
                    ('CATPCA', PCA(scaling=True, scores=True)),
                    ('HGBT_Classifier', HybridGradientBoostingClassifier(
                                            n_estimators=4, split_threshold=0,
                                            learning_rate=0.5, fold_num=5,
                                            max_depth=6))])

In [ ]:
diabetes_full, diabetes_train, diabetes_test, _ = DataSets.load_diabetes_data(connection_context)

In [ ]:
param = {'CATPCA__key': 'ID', 'CATPCA__label': 'CLASS'}
my_pipeline.fit(diabetes_train, fit_params=param, generate_json_pipeline=True)


In [ ]:
my_pipeline.pipeline

In [ ]:
from hana_ml.algorithms.pal.auto_ml import AutomaticClassification

auto_c = AutomaticClassification()

In [ ]:
auto_c.disable_workload_class_check()
auto_c.fit(diabetes_train, pipeline=my_pipeline.pipeline)

#### EDA for time seires data

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm
from scipy.linalg import cholesky
import numpy as np
from numpy.random import rand

num_samples = 600
S1 = 12
S2 = 100

np.random.seed(seed=2334)

x1 = norm.rvs(loc=0, scale=1, size=(1, num_samples))[0]
x2 = norm.rvs(loc=0, scale=1, size=(1, num_samples))[0]
x3 = norm.rvs(loc=0, scale=1, size=(1, num_samples))[0]
x4 = norm.rvs(loc=0, scale=1, size=(1, num_samples))[0]

std_m = np.array([
    [6.8, 0, 0, 0],
    [0, 1.4, 0, 0],
    [0, 0, 1.4, 0],
    [0, 0, 0, 2.9]
])

# specify desired correlation
corr_m = np.array([
    [1, .35, 0.33, 0.78],
    [.35, 1, 0.90, 0.28],
    [.33, 0.90, 1, 0.27],
    [.78, 0.28, 0.27, 1]
])

# calc desired covariance (vc matrix)
cov_m = np.dot(std_m, np.dot(corr_m, std_m))
L = cholesky(cov_m, lower=True)
corr_data = np.dot(L, [x1, x2, x3, x4]).T

beta=np.array([-3.49, 13, 13, 0.0056])
omega1 = 2*np.pi/S1
omega2 = 2*np.pi/S2
timestamp = np.array([i for i in range(num_samples)])
y1 = np.multiply(50*rand(num_samples), 20*rand(1)*np.cos(omega1*timestamp)) \
+ np.multiply(32*rand(num_samples), 30*rand(1)*np.cos(3*omega1*timestamp)) \
+ np.multiply(rand(num_samples), rand(1)*np.sin(omega2*timestamp)) 

y2 = np.multiply(rand(num_samples), timestamp)
y3 = corr_data.dot(beta.T)
y = y1 + y2 + y3

In [ ]:
dates = pd.date_range('2018-01-01', '2019-08-23',freq='D')
data_additive = {'ID':dates, 'Y':y, 'X1':corr_data[:,0], 'X2':corr_data[:,1], 'X3':corr_data[:,2], 'X4':corr_data[:,3]}
data = pd.DataFrame(data_additive)
df_acf =dataframe.create_dataframe_from_pandas(connection_context, data, table_name='#PAL_ACF_PLOT', force=True)

In [ ]:
from hana_ml.visualizers.eda import plot_acf, plot_pacf, plot_moving_average, quarter_plot
from hana_ml.visualizers.eda import plot_rolling_stddev, seasonal_plot, timeseries_box_plot, plot_seasonal_decompose

In [ ]:
plot_acf(df_acf, key="ID", col="Y")

In [ ]:
plot_pacf(df_acf.set_index("ID"), col="X1")

In [ ]:
fig, ax = plt.subplots(figsize=(15, 10))
ax = plot_moving_average(df_acf.set_index("ID"), col="Y", rolling_window=-3, ax=ax)


In [ ]:
fig, ax = plt.subplots(figsize=(15, 10))
ax = plot_rolling_stddev(df_acf.set_index("ID"), col="Y", rolling_window=10, ax=ax)


In [ ]:
df_acf.generate_feature(targets=["Y"], order_by="ID", trans_func="AVG", rolling_window=1).collect()

In [ ]:
df_acf.generate_feature(targets=["Y"], order_by="ID", trans_func="STDDEV", rolling_window=-3).collect()

In [ ]:
df_acf.generate_feature(targets=["Y"], order_by="ID", trans_func="STDDEV", rolling_window="inf").collect()

In [ ]:
df_acf.generate_feature(targets=["Y"], order_by="ID", trans_func="CORR", second_targets=["X1"], rolling_window="-Inf").collect()

In [ ]:
df_acf.corr("Y", "X1").collect()

In [ ]:
df_acf.collect()

In [ ]:
seasonal_plot(df_acf.set_index("ID"), col="Y")

In [ ]:
timeseries_box_plot(df_acf, "Y", key="ID", cycle="YEAR")

In [ ]:
timeseries_box_plot(df_acf, "Y", key="ID", cycle="MONTH")

In [ ]:
fig, ax = plt.subplots(figsize=(15, 10))
timeseries_box_plot(df_acf, "Y", key="ID", cycle="QUARTER", ax=ax)

In [ ]:
fig, axes = plt.subplots(3, 1, figsize=(12, 8))
fig.suptitle('Seasonal Decompose', fontsize=16)
plot_seasonal_decompose(df_acf, col="Y", axes=axes)


In [ ]:
fig, ax = plt.subplots(figsize=(15, 10))
fig.suptitle('Quarter Plot', fontsize=16)
ax = quarter_plot(df_acf, col="Y", key="ID", ax=ax)


#### Missing Value Handling in Unified Interfaces

In [2]:
import pandas as pd
iris = pd.read_csv('../datasets/iris.csv', header=None)
iris.columns = ['V1', 'V2', 'V3', 'V4', 'CLASS']

In [3]:
## Manually creating some missing values in iris data
iris_dict = dict(ID=range(iris.shape[0]))
import numpy as np
for col in iris.columns[:-1]:
    x = np.random.rand(iris.shape[0])
    iris_dict[col] = [y if z > 0.1 else None for (y, z) in zip(iris[col], x)]
iris_dict['CLASS'] = iris['CLASS']

In [4]:
iris_data = pd.DataFrame(iris_dict)

In [5]:
from hana_ml.dataframe import create_dataframe_from_pandas
iris_df = create_dataframe_from_pandas(connection_context, iris_data, 'IRIS_MISS_FULL', force=True)

100%|████████████████████████████████████████████| 1/1 [00:00<00:00,  1.97it/s]


In [6]:
from hana_ml.algorithms.pal.unified_classification import UnifiedClassification
uc_mlogr = UnifiedClassification(func='LogisticRegression', multi_class=True)
uc_mlogr.fit(iris_df, key='ID', impute=True, strategy='most_frequent-mean',
             strategy_by_col=[('V2', 'median')])

In [7]:
uc_mlogr.score(iris_df.head(50), key='ID', impute=True, strategy='most_frequent-mean',
               strategy_by_col=[('V2', 'median')])#

(<hana_ml.dataframe.DataFrame at 0x22eb7598c48>,
 <hana_ml.dataframe.DataFrame at 0x22eb7593188>)

In [8]:
boston_housing = pd.read_csv('../datasets/boston-house-prices.csv', header=None)

In [9]:
boston_housing.columns = ['V' + str(i) for i in range(boston_housing.shape[1]-1)] + ['MEDV']

In [11]:
rnd = np.random.rand(boston_housing.shape[0])
boston_housing['V0'] = [y if z > 0.1 else None for y, z in zip(boston_housing['V0'], rnd)]

In [13]:
boston_df = create_dataframe_from_pandas(connection_context,
                                         boston_housing,
                                         'BOSTON_HOUSE_MISS_FULL',
                                         force=True)

100%|████████████████████████████████████████████| 1/1 [00:00<00:00,  1.67it/s]


In [14]:
from hana_ml.algorithms.pal.unified_regression import UnifiedRegression
ur_lr = UnifiedRegression(func='LinearRegression')
ur_lr.fit(boston_df, label='MEDV', impute=True, strategy_by_col=[('V0', 'median')])

In [15]:
ur_lr.score(boston_df.head(50).add_id('ID'), key='ID', impute=True)

(<hana_ml.dataframe.DataFrame at 0x22eb85af188>,
 <hana_ml.dataframe.DataFrame at 0x22eb85adbc8>)

#### GARCH Enhancement

In [4]:
import numpy as np
import pandas as pd
npt = 20
np.random.seed(2022)
ts_data = pd.DataFrame(dict(TIME = range(npt),
                            X = 2*(np.random.rand(npt) - 0.5)))

In [7]:
from hana_ml.dataframe import create_dataframe_from_pandas
ts_df = create_dataframe_from_pandas(connection_context, ts_data, 'GARCH_TS_SIMU_DATA', force=True)

100%|████████████████████████████████████████████| 1/1 [00:00<00:00,  2.15it/s]


In [9]:
from hana_ml.algorithms.pal.tsa.garch import GARCH

In [12]:
gch = GARCH(model_type='igarch')#integrated GARCH
gch.fit(data = ts_df, key='TIME')
res = gch.predict(horizon=3)

In [14]:
res[0].collect()

,STEP,VARIANCE,RESIDUAL
0,1,0.399849,None
1,2,0.399849,None
2,3,0.399849,None


In [15]:
gch = GARCH(model_type='tgarch')#threshold GARCH
gch.fit(data = ts_df, key='TIME')
res = gch.predict(horizon=3)

In [17]:
res[0].collect()

,STEP,VARIANCE,RESIDUAL
0,1,0.383388,None
1,2,0.407847,None
2,3,0.409407,None


In [21]:
gch = GARCH(model_type='egarch')#exponential GARCH
gch.fit(data = ts_df, key='TIME')

#### Local Intepretability for KNN

In [2]:
from hana_ml.algorithms.pal.utility import DataSets

In [8]:
iris_full,_,_,_ = DataSets.load_iris_data(connection_context)

Table IRIS_DATA_FULL_TBL exists.


In [10]:
from hana_ml.algorithms.pal.partition import train_test_val_split

In [22]:
iris_train, iris_test, _ = train_test_val_split(iris_full,
                                                id_column='ID',
                                                random_seed=2022,
                                                partition_method='stratified',
                                                stratified_column='SPECIES',
                                                training_percentage=0.8,
                                                testing_percentage=0.2,
                                                validation_percentage=0.0)

In [23]:
from hana_ml.algorithms.pal.neighbors import KNNClassifier
knn = KNNClassifier(n_neighbors=3, metric='euclidean')
knn.fit(iris_train, key='ID', label='SPECIES')

In [25]:
res = knn.predict(iris_test, key='ID',
                  features=iris_test.columns[1:-1],
                  interpret=True)# set 'interpret' parameter to be True

In [27]:
res[0].collect()#view prediction result together with local interpretability(reason code)

,ID,TARGET,REASON_CODE
0,5,Iris-setosa,"[{""attr"":""PETALLENGTHCM"",""val"":0.6080661302641..."
1,6,Iris-setosa,"[{""attr"":""PETALLENGTHCM"",""val"":0.6366492571233..."
2,8,Iris-setosa,"[{""attr"":""PETALLENGTHCM"",""val"":0.6387009267078..."
3,17,Iris-setosa,"[{""attr"":""PETALLENGTHCM"",""val"":0.6366287059064..."
4,20,Iris-setosa,"[{""attr"":""PETALLENGTHCM"",""val"":0.6100976898526..."
5,22,Iris-setosa,"[{""attr"":""PETALLENGTHCM"",""val"":0.6329967936354..."
6,23,Iris-setosa,"[{""attr"":""PETALLENGTHCM"",""val"":0.6101638094630..."
7,27,Iris-setosa,"[{""attr"":""PETALLENGTHCM"",""val"":0.6339550745825..."
8,28,Iris-setosa,"[{""attr"":""PETALLENGTHCM"",""val"":0.6366496435427..."
9,32,Iris-setosa,"[{""attr"":""PETALLENGTHCM"",""val"":0.6239599876740..."


#### Croston TSB

- support both single and massive mode

In [3]:
from hana_ml.algorithms.pal.tsa.exponential_smoothing import CrostonTSB
import pandas as pd
from hana_ml.dataframe import create_dataframe_from_pandas
data = [['100', 1,   3],
        ['100', 2,   6],
        ['100', 3,   4],
        ['100', 4,   0],
        ['100', 5,   0],
        ['100', 6,   0],
        ['100', 7,   0],
        ['100', 8,   0],
        ['100', 9,   0],
        ['100', 10,  16],
        ['200', 1,   5],
        ['200', 2,   0],
        ['200', 3,   0],
        ['200', 4,   3],
        ['200', 5,   0],
        ['200', 6,   0],
        ['200', 7,   0],
        ['200', 8,   0],
        ['200', 9,   8],
        ['200', 10,  0]]
col_name = ["GROUP_ID", "ID", "y"]
df_m = pd.DataFrame(data=data, columns=col_name)
df_m = create_dataframe_from_pandas(connection_context=connection_context, 
                                    pandas_df=df_m,
                                    table_name='DATA_MASSIVE_CROSTON_NOTEBOOK', 
                                    force=True, 
                                    replace=True)
mcr = CrostonTSB(massive=True,
                 group_params= {'100': {'accuracy_measure':'MAPE', 'expost_flag' :False, 'forecast_num':1},
                                '200': {'accuracy_measure':'MSE', 'expost_flag' :True, 'forecast_num':1}})
res = mcr.fit_predict(df_m, key='ID', endog='y', group_key="GROUP_ID")
print(res.collect())
print(mcr.forecast_.collect())
print(mcr.stats_.collect())
print(mcr.metrics_.collect())
print(mcr.error_msg_.collect())

100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  1.67it/s]


   GROUP_ID  ID  OUTPUT_VALUE
0       100  11          0.00
1       200   2          0.00
2       200   3          0.00
3       200   4          5.30
4       200   5          0.00
5       200   6          0.00
6       200   7          0.00
7       200   8          0.00
8       200   9          5.07
9       200  10          0.00
10      200  11          0.00
   GROUP_ID  ID  OUTPUT_VALUE
0       100  11          0.00
1       200   2          0.00
2       200   3          0.00
3       200   4          5.30
4       200   5          0.00
5       200   6          0.00
6       200   7          0.00
7       200   8          0.00
8       200   9          5.07
9       200  10          0.00
10      200  11          0.00
  GROUP_ID STAT_NAME  STAT_VALUE
0      100      MAPE    0.157651
1      200       MSE    1.541656
  GROUP_ID                  NAME     VALUE
0      100       DEMAND_FORECAST  7.421425
1      100  PROBABILITY_FORECAST  0.345845
2      200       DEMAND_FORECAST  5.363000
3      20

### Massive Mode Support Enhancement

- supported in ARIMA, Auto ARIMA, Additive Model Forecast, UnifiedClassification, UnifiedRegression


In [7]:
# Take UnifiedClassification as an example
data = [['1', 1, 'Sunny', 75, 70.0, 'Yes', 'Play'],
        ['1', 2, 'Sunny', 80, 90.0, 'Yes', 'Do not Play'],
        ['1', 3, 'Sunny', 85, 91.0, 'No', 'Do not Play'],
        ['1', 4, 'Sunny', 72, 95.0, 'No', 'Do not Play'],
        ['1', 5, 'Sunny', 73, 70.0, 'No', 'Play'],
        ['1', 6, 'Overcast', 72.0, 90, 'Yes', 'Do not Play'],
        ['1', 7, 'Overcast', 83.0, 78, 'No', 'Play'],
        ['1', 8, 'Overcast', 64.0, 65, 'Yes', 'Do not Play'],
        ['1', 9, 'Overcast', 81.0, 75, 'No', 'Play'],
        ['1', 10, 'Rain', 71, 80.0, 'Yes', 'Play'],
        ['1', 11, 'Rain', 65, 70.0, 'Yes', 'Do not Play'],
        ['1', 12, 'Rain', 75, 80.0, 'No', 'Play'],
        ['1', 13, 'Rain', 68, 80.0, 'No', 'Play'],
        ['1', 14, 'Rain', 70, 96.0, 'No', 'Play'],
        ['2', 1, 'Overcast', 68, 70.0, 'No', 'Play'],
        ['2', 2, 'Overcast', 80, 90.0, 'No', 'Do not Play'],
        ['2', 3, 'Overcast', 85, 91.0, 'Yes', 'Do not Play'],
        ['2', 4, 'Overcast', 78, 95.0, 'Yes', 'Do not Play'],
        ['2', 5, 'Overcast', 73, 70.0, 'Yes', 'Play'],
        ['2', 6, 'Sunny', 68.0, 90, 'No', 'Do not Play'],
        ['2', 7, 'Sunny', 83.0, 78, 'Yes', 'Play'],
        ['2', 8, 'Sunny', 64.0, 65, 'Yes', 'Do not Play'],
        ['2', 9, 'Sunny', 81.0, 75, 'No', 'Play'],
        ['2', 10, 'Rain', 65, 80.0, 'No', 'Play'],
        ['2', 11, 'Rain', 65, 70.0, 'No', 'Do not Play'],
        ['2', 12, 'Rain', 85, 80.0, 'Yes', 'Play'],
        ['2', 13, 'Rain', 68, 80.0, 'Yes', 'Play'],
        ['2', 14, 'Rain', 70, 96.0, 'Yes', 'Play']]
col_name = ["GROUP_ID", "ID", "OUTLOOK", "TEMP", "HUMIDITY", "WINDY", "CLASS"]
df_m = pd.DataFrame(data=data,columns=col_name)
df = create_dataframe_from_pandas(connection_context=connection_context, 
                                  pandas_df=df_m,
                                  table_name='DATA_UNI_MASSIVE_TBL_NOTEBOOK', 
                                  force=True, 
                                  replace=True)

100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  1.35it/s]


In [12]:
from hana_ml.algorithms.pal.unified_classification import UnifiedClassification
uc = UnifiedClassification(func='logisticregression',    
                           multi_class=True,
                           massive=True,
                           max_iter=10,
                           group_params={'1': {'solver': 'auto'}})
uc.fit(data=df,
       key='ID',
       features=["OUTLOOK" ,"TEMP", "HUMIDITY","WINDY"],
       label="CLASS",
       group_key="GROUP_ID",
       background_size=4,
       group_params={'1': {'background_random_state':2}})

res, error=uc.predict(df.deselect("CLASS"), 
                      key='ID', 
                      group_key='GROUP_ID', 
                      features=["OUTLOOK" ,"TEMP", "HUMIDITY","WINDY"],                        
                      verbose=True,
                      group_params={'1': {'thread_ratio':1}})
print(res.head(3).collect())
print(error.collect())

res = uc.score(df, 
               key='ID', 
               group_key='GROUP_ID', 
               features=["OUTLOOK" ,"TEMP", "HUMIDITY","WINDY"], 
               label= "CLASS",
               thread_ratio=0.5,
               group_params={'1': {'thread_ratio':1}})
print(res[0].head(3).collect())
print(res[1].head(3).collect())
print(res[2].head(3).collect())
print(res[3].head(3).collect())
print(res[4].collect())

  GROUP_ID  ID SCORE  CONFIDENCE REASON_CODE
0        1  14  Play    0.606915        None
1        1  13  Play    0.954580        None
2        1  12  Play    0.988835        None
Empty DataFrame
Columns: [GROUP_ID, ERROR_TIMESTAMP, ERRORCODE, MESSAGE]
Index: []
  GROUP_ID  ID        SCORE  CONFIDENCE REASON_CODE
0        1   1  Do not Play    0.623109        None
1        1   2  Do not Play    0.962771        None
2        1   3         Play    0.641685        None
  GROUP_ID  STAT_NAME          STAT_VALUE   CLASS_NAME
0        1        AUC  0.9132653061224489         None
1        1     RECALL  0.6666666666666666  Do not Play
2        1  PRECISION                 0.8  Do not Play
  GROUP_ID ACTUAL_CLASS PREDICTED_CLASS  COUNT
0        1  Do not Play     Do not Play      4
1        1  Do not Play            Play      2
2        1         Play     Do not Play      1
  GROUP_ID             NAME    X    Y
0        1  RANDOM_CUMGAINS  0.0  0.0
1        1  RANDOM_CUMGAINS  1.0  1.0
2      

In [ ]:
full_set, training_set, validation_set, test_set = DataSets.load_boston_housing_data(connection_context)

In [ ]:
features=['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', 'DIS', 'RAD', 'TAX', 'PTRATIO', 'BLACK', 'LSTAT']
label='MEDV'
# Cast to correct types so PAL can consume it.
dfts = training_set.cast(['CRIM', "ZN", "INDUS", "NOX", "RM", "AGE", "DIS", "PTRATIO", "BLACK", "LSTAT", "MEDV"], "DOUBLE")
dfts = dfts.cast(["CHAS", "RAD", "TAX"], "INTEGER")
dfts = dfts.to_head("ID")
dfts.head(5).collect()

In [ ]:
# Useful for multiple runs (only doing one run in this sample notebook)
import mlflow
df = dfts.select(features, label)
model = UnifiedRegression(func='RandomForest')
runid = None
with mlflow.start_run() as run:
    par_params = dict(partition_method='random',
                      training_percent=0.7,
                      partition_random_state=2,
                      output_partition_result=True)
    model.enable_mlflow_autologging()
    model.fit(df, features=features, label=label, **par_params)
    runid = run.info.run_id


In [ ]:
from hana_ml.model_storage import ModelStorage
mymodel = ModelStorage.load_mlflow_model(connection_context=connection_context, model_uri='runs:/{}/model'.format(runid))
mymodel